In [108]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from firebase import firebase
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

# Firebase URL
firebase_url = 'https://hw2-tiger-default-rtdb.europe-west1.firebasedatabase.app/'

# Initialize the connection to Firebase
FBconn = firebase.FirebaseApplication(firebase_url, None)

# Fetch data from Firebase
def fetch_data_from_firebase():
    result = FBconn.get('/HW2-Tiger/', None)
    if result:
        data = [value for value in result.values()]
        return data
    else:
        return []

# Email configuration
EMAIL_ADDRESS = 'uriziv123654@gmail.com'
EMAIL_PASSWORD = 'yqce zdce tqrn wslc'  # Use the generated app password
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587

# Send email with attachment
def send_email_with_attachment(subject, body, attachment_path, recipient_email):
    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = recipient_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'html'))

    attachment = MIMEBase('application', 'octet-stream')
    with open(attachment_path, 'rb') as file:
        attachment.set_payload(file.read())
    encoders.encode_base64(attachment)
    attachment.add_header('Content-Disposition', f'attachment; filename={os.path.basename(attachment_path)}')
    msg.attach(attachment)

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.send_message(msg)

# Title and Welcome Message
now = datetime.now()
current_time_date = now.strftime("%Y-%m-%d %H:%M:%S")
title = widgets.HTML(value=f"""
    <div style='background: linear-gradient(to right, #6EC373, #ffffff); padding: 20px; border-radius: 10px; color: black;'>
        <h1 style='color: black'>Welcome to OnShape, Amir Cohen</h1>
        <h2 style='color: black'>{current_time_date}</h2>
    </div>
""")

# Set a greenish color palette
green_palette = sns.color_palette("Greens", as_cmap=True)

# Function to create the main dashboard menu
def create_main_dashboard():
    dashboard_buttons = [
        widgets.Button(description='Actions by User', layout=widgets.Layout(width='200px', height='100px')),
        widgets.Button(description='Actions by Document', layout=widgets.Layout(width='200px', height='100px')),
        widgets.Button(description='Number of Actions Over Time', layout=widgets.Layout(width='200px', height='100px')),
        widgets.Button(description='Actions by Tab', layout=widgets.Layout(width='200px', height='100px')),
    ]

    for i in range(len(dashboard_buttons)):
        dashboard_buttons[i].style.css = {'color': 'black'}

    # Style buttons
    button_style = "background-color: #6EC373; color: #ffffff; border-radius: 10px;"
    for button in dashboard_buttons:
        button.style = {'button_color': '#023407', 'font_size': '16px', 'color': 'black', 'border_radius': '20px'}

    dashboard_grid = widgets.GridBox(
        children=dashboard_buttons,
        layout=widgets.Layout(
            width='100%',
            grid_template_columns='repeat(2, 200px)',
            grid_template_rows='repeat(2, 100px)',
            grid_gap='10px'
        )
    )

    def go_to_dashboard(button):
        display_main_dashboard()

    for button in dashboard_buttons:
        button.on_click(go_to_dashboard)

    return dashboard_grid

# Placeholder for plots
plot_output = widgets.Output()

# Function to create individual pages
def create_page(title_text, additional_widgets=None):
    page_title = widgets.HTML(value=f"""
        <div style='background: linear-gradient(to right, #6EC373, #ffffff); padding: 10px; border-radius: 10px;'>
            <h1 style='color: black'>{title_text}</h1>
        </div>
    """)
    back_button = widgets.Button(description='Back to Dashboard', layout=widgets.Layout(width='200px', height='50px'))
    back_button.style = {'button_color': '#023407', 'font_size': '16px', 'border_radius': '10px'}

    def back_to_dashboard(b):
        plot_output.clear_output()  # Clear plot output when navigating back to dashboard
        display_main_dashboard()

    back_button.on_click(back_to_dashboard)

    children = [page_title, back_button]
    if additional_widgets:
        children.extend(additional_widgets)

    page = widgets.VBox(children)
    return page

# Utilities methods and functions
def truncate_labels(labels, max_length=25):
    truncated = [label if len(label) <= max_length else label[:max_length-3] + "..." for label in labels]
    return truncated

# Plotting functions with save functionality
def plot_actions_by_user(save_path='actions_by_user.png'):
    data = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the figsize for better layout
            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, df['User'].nunique()))  # Apply the color palette

            user_counts = df['User'].value_counts()
            user_counts.plot(kind='pie', ax=ax, colors=colors, autopct='%1.1f%%', startangle=90)
            ax.set_title('Number of Actions by User', fontsize=16)
            ax.set_ylabel('')  # Remove y-label for a cleaner look
            ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_actions_by_document(save_path='actions_by_document.png'):
    data = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the figsize for better layout
            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, df['Document'].nunique()))  # Apply the color palette

            document_counts = df['Document'].value_counts()
            truncated_labels = truncate_labels(document_counts.index)

            document_counts.index = truncated_labels  # Update the labels with truncated versions

            document_counts.plot(kind='bar', ax=ax, color=colors)
            ax.set_title('Number of Actions by Document', fontsize=16)
            ax.set_xlabel('Document', fontsize=14)
            ax.set_ylabel('Number of Actions', fontsize=14)
            plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate and adjust label font size
            plt.yticks(fontsize=12)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_number_of_actions_over_time(save_path='number_of_actions_over_time.png'):
    data = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        df['Time'] = pd.to_datetime(df['Time'], errors='coerce')  # Coerce errors to NaT
        df = df.dropna(subset=['Time'])  # Drop rows with NaT in 'Time'
        df = df[df['Time'] > datetime(2000, 1, 1)]  # Remove any erroneous old dates

        df_resampled = df.resample('D', on='Time').size()

        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(8, 4))  # Adjust the figsize for better layout
            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, len(df_resampled)))  # Apply the color palette
            df_resampled.plot(kind='line', ax=ax, color=colors[0])  # Use the first color for the line
            ax.set_title('Number of Actions Over Time', fontsize=16)
            ax.set_xlabel('Time', fontsize=14)
            ax.set_ylabel('Number of Actions', fontsize=14)

            # Improve x-axis date formatting
            ax.xaxis.set_major_locator(mdates.MonthLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
            plt.xticks(rotation=90, ha='right', fontsize=12)

            plt.yticks(fontsize=12)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

def plot_actions_by_tab(document_name, save_path='actions_by_tab.png'):
    data = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        df = df[df['Document'] == document_name].copy()
        df.loc[df['Tab'] == 'N/A', 'Tab'] = 'Other'
        with plot_output:
            plot_output.clear_output()
            fig, ax = plt.subplots(figsize=(8, 4))  # Adjust the figsize for better layout

            tab_counts = df['Tab'].value_counts()
            truncated_labels = truncate_labels(tab_counts.index)

            tab_counts.index = truncated_labels  # Update the labels with truncated versions

            colors = sns.color_palette("Greens", as_cmap=True)(np.linspace(0.5, 1, tab_counts.nunique()))  # Apply the color palette

            tab_counts.plot(kind='bar', ax=ax, color=colors)
            ax.set_title(f'Number of Actions by Tab for {document_name}', fontsize=16)
            ax.set_xlabel('Tab', fontsize=14)
            ax.set_ylabel('Number of Actions', fontsize=14)
            plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate and adjust label font size
            plt.yticks(fontsize=12)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.show()
            fig.savefig(save_path, format='png')  # Save the plot as an image

# Function to create the send email button and email input field
def create_send_email_button(image_path, subject):
    send_email_button = widgets.Button(
        description='Send Email',
        layout=widgets.Layout(width='200px', height='50px'),
        style=widgets.ButtonStyle(button_color='#023407')
    )

    email_input = widgets.Text(
        description='Email:',
        layout=widgets.Layout(width='300px')
    )

    def on_send_email_button_clicked(b):
        recipient_email = email_input.value
        if recipient_email:
            body = f"<p>Please find the attached graph image for {subject}.</p>"
            send_email_with_attachment(subject, body, image_path, recipient_email)
            with plot_output:
                display(widgets.HTML(f"<p>Email sent to {recipient_email} with the attached graph image for {subject}.</p>"))
        else:
            with plot_output:
                display(widgets.HTML("<p>Please enter a valid email address.</p>"))

    send_email_button.on_click(on_send_email_button_clicked)
    return widgets.VBox([email_input, send_email_button])

# Function to display the plot and send email button
def display_plot_with_send_email(plot_function, save_path, subject):
    clear_output()
    plot_function(save_path)  # Create the plot and save it as an image
    send_email_controls = create_send_email_button(save_path, subject)
    page = create_page(subject, additional_widgets=[send_email_controls])
    display(page)
    display(plot_output)

# Function to display the dropdown menu for selecting a document
def display_document_selection():
    data = fetch_data_from_firebase()
    if data:
        df = pd.DataFrame(data)
        # Remove NaN values before creating the list of options
        documents = ['Choose...'] + list(df['Document'].dropna().unique())
        document_selector = widgets.Dropdown(
            options=documents,
            description='Select Document:',
            layout=widgets.Layout(width='400px')
        )

        def on_document_selected(change):
            if change['type'] == 'change' and change['name'] == 'value':
                if change['new'] == 'Choose...':
                    plot_output.clear_output()
                else:
                    display_plot_with_send_email(lambda path: plot_actions_by_tab(change['new'], path), 'actions_by_tab.png', f'Actions by Tab for {change["new"]}')

        document_selector.observe(on_document_selected)

        additional_widgets = [document_selector]
        clear_output()
        page = create_page('Actions by Tab', additional_widgets)
        display(page)
        display(plot_output)

# Functions to display different pages
def display_actions_by_user():
    display_plot_with_send_email(plot_actions_by_user, 'actions_by_user.png', 'Actions by User')

def display_actions_by_document():
    display_plot_with_send_email(plot_actions_by_document, 'actions_by_document.png', 'Actions by Document')

def display_number_of_actions_over_time():
    display_plot_with_send_email(plot_number_of_actions_over_time, 'number_of_actions_over_time.png', 'Number of Actions Over Time')

def display_actions_by_tab():
    clear_output()
    display(title)
    display_document_selection()

# Display the main dashboard
def display_main_dashboard():
    clear_output()
    plot_output.clear_output()  # Clear plot output when navigating to main dashboard
    dashboard_grid = create_main_dashboard()
    dashboard_grid.children[0].on_click(lambda b: display_actions_by_user())
    dashboard_grid.children[1].on_click(lambda b: display_actions_by_document())
    dashboard_grid.children[2].on_click(lambda b: display_number_of_actions_over_time())
    dashboard_grid.children[3].on_click(lambda b: display_actions_by_tab())
    display(title, dashboard_grid)

# Initial display
display_main_dashboard()


HTML(value="\n    <div style='background: linear-gradient(to right, #6EC373, #ffffff); padding: 20px; border-r…

GridBox(children=(Button(description='Actions by User', layout=Layout(height='100px', width='200px'), style=Bu…